In [ ]:
!pip install POT

In [ ]:
import pandas as pd
import numpy as np
import random
import pickle
import urllib.request
import networkx as nx
import ot

In [ ]:
url = "https://raw.githubusercontent.com/potentialreviewer/Optimal-SNA/main/data/500N-KP-Crowd_dtm_dict.pkl"
file_name = "500N-KP-Crowd_dtm_dict.pkl"

urllib.request.urlretrieve(url, file_name)

with open("500N-KP-Crowd_dtm_dict.pkl", "rb") as f:
    DTM_dict = pickle.load(f)

In [ ]:
class TTMBuilder:
    def __init__(self, dtm_dict, dataset_name, method, zeta):
        self.dtm_dict = dtm_dict
        self.dataset_name = dataset_name
        self.method = method
        self.zeta = zeta

        self.key = f"{dataset_name}_{method}_zeta{zeta}"

        self.dtm = dtm_dict[self.key]
        self.ttm = None

    def build_CF_ttm(self):
        binary_dtm = (self.dtm > 0).astype(int)
        cf_matrix = binary_dtm.T.dot(binary_dtm)

        self.ttm = cf_matrix
        return self.ttm

    def build_Dice_ttm(self):
        cf_matrix = self.build_CF_ttm()
        diag = np.diag(cf_matrix)

        i_diag = diag[:, np.newaxis]
        j_diag = diag[np.newaxis, :]
        dice_matrix = 2 * cf_matrix / (i_diag + j_diag)
        np.fill_diagonal(dice_matrix.values, 1.0)

        self.ttm = pd.DataFrame(dice_matrix, index=cf_matrix.index, columns=cf_matrix.columns)
        return self.ttm

    def build_Jaccard_ttm(self):
        cf_matrix = self.build_CF_ttm()
        diag = np.diag(cf_matrix)

        i_diag = diag[:, np.newaxis]
        j_diag = diag[np.newaxis, :]
        denominator = i_diag + j_diag - cf_matrix
        jaccard_matrix = cf_matrix / denominator
        np.fill_diagonal(jaccard_matrix.values, 1.0)

        self.ttm = pd.DataFrame(jaccard_matrix, index=cf_matrix.index, columns=cf_matrix.columns)
        return self.ttm

    def build_Cosine_ttm(self):
        term_vectors = self.dtm.T
        similarity_matrix = term_vectors.dot(term_vectors.T)
        norms = (term_vectors ** 2).sum(axis=1) ** 0.5
        self.ttm = similarity_matrix.divide(norms, axis=0).divide(norms, axis=1)
        return self.ttm

    def build_Euclidean_ttm(self):
        term_vectors = self.dtm.T.values
        term_names = self.dtm.columns

        dist_matrix = np.zeros((len(term_vectors), len(term_vectors)))
        for i in range(len(term_vectors)):
            for j in range(i + 1, len(term_vectors)):
                dist = np.sqrt(np.sum((term_vectors[i] - term_vectors[j]) ** 2))
                dist_matrix[i, j] = dist
                dist_matrix[j, i] = dist

        max_dist = np.max(dist_matrix)
        similarity_matrix = 1 - (dist_matrix / max_dist)
        self.ttm = pd.DataFrame(similarity_matrix, index=term_names, columns=term_names)
        return self.ttm

    def build_Manhattan_ttm(self):
        term_vectors = self.dtm.T.values
        term_names = self.dtm.columns

        dist_matrix = np.zeros((len(term_vectors), len(term_vectors)))
        for i in range(len(term_vectors)):
            for j in range(i + 1, len(term_vectors)):
                dist = np.sum(np.abs(term_vectors[i] - term_vectors[j]))
                dist_matrix[i, j] = dist
                dist_matrix[j, i] = dist

        max_dist = np.max(dist_matrix)
        similarity_matrix = 1 - (dist_matrix / max_dist)
        self.ttm = pd.DataFrame(similarity_matrix, index=term_names, columns=term_names)
        return self.ttm

    def build_DMD_ttm(self):
        dtm = self.dtm.values
        m, n = dtm.shape

        cost_matrix = np.zeros((m, m))
        for k in range(m):
            for l in range(k + 1, m):
                dist = np.sqrt(np.sum((dtm[k] - dtm[l]) ** 2))
                cost_matrix[k, l] = dist
                cost_matrix[l, k] = dist

        term_vectors = dtm.T

        dmd_matrix = np.full((n, n), np.nan)

        for i in range(n):
            for j in range(i, n):
                p = term_vectors[i]
                q = term_vectors[j]

                if np.sum(p) == 0 or np.sum(q) == 0:
                    continue
                else:
                    p_norm = p / np.sum(p)
                    q_norm = q / np.sum(q)
                    dmd = ot.emd2(p_norm, q_norm, cost_matrix)

                    dmd_matrix[i, j] = dmd
                    dmd_matrix[j, i] = dmd

        finite_mask = ~np.isnan(dmd_matrix)
        max_dist = np.max(dmd_matrix[finite_mask])
        dmd_matrix[np.isnan(dmd_matrix)] = max_dist

        sim_matrix = 1 - (dmd_matrix / max_dist)

        self.ttm = pd.DataFrame(sim_matrix, index=self.dtm.columns, columns=self.dtm.columns)

        return self.ttm

In [ ]:
def generate_ttm_dict(dtm_dict, edge_methods):

    ttm_dict = {}

    for key in dtm_dict:
        prefix, zeta = key.split('_zeta')
        dataset, method = prefix.split('_')

        builder = TTMBuilder(dtm_dict, dataset, method, zeta)

        if dataset not in ttm_dict:
            ttm_dict[dataset] = {}
        if method not in ttm_dict[dataset]:
            ttm_dict[dataset][method] = {}
        if zeta not in ttm_dict[dataset][method]:
            ttm_dict[dataset][method][zeta] = {}

        for em in edge_methods:
            func = getattr(builder, f"build_{em}_ttm")
            ttm_dict[dataset][method][zeta][em] = func()

    return ttm_dict

In [ ]:
edge_methods = ["CF", "Dice", "Jaccard", "Cosine", "Euclidean", "Manhattan", "DMD"]
TTM_dict = generate_ttm_dict(DTM_dict, edge_methods)

In [ ]:
with open("500N-KP-Crowd_ttm_dict.pkl", "wb") as f:
    pickle.dump(TTM_dict, f)

In [ ]:
with open("500N-KP-Crowd_ttm_dict.pkl", "rb") as f:
    TTM_dict = pickle.load(f)

In [ ]:
def build_graph_from_ttm(ttm_df, omega=0.1):

    G = nx.Graph()
    terms = ttm_df.index.tolist()
    G.add_nodes_from(terms)

    edges = []
    for i in range(1, len(terms)):
        for j in range(0, i):
            weight = ttm_df.iat[i, j]
            if weight > 0:
                edges.append((terms[i], terms[j], weight))

    edges.sort(key=lambda x: x[2], reverse=True)
    k = int(len(edges) * omega)
    top_edges = edges[:k]

    for u, v, w in top_edges:
        G.add_edge(u, v, weight=w)

    return G

In [ ]:
def generate_graph_dict_from_ttm_dict(ttm_dict, omega=0.1):

    graph_dict = {}

    for dataset in ttm_dict:
        graph_dict[dataset] = {}
        for method in ttm_dict[dataset]:
            graph_dict[dataset][method] = {}
            for zeta in ttm_dict[dataset][method]:
                graph_dict[dataset][method][zeta] = {}
                for edge_method in ttm_dict[dataset][method][zeta]:
                    ttm = ttm_dict[dataset][method][zeta][edge_method]
                    G = build_graph_from_ttm(ttm, omega=omega)
                    graph_dict[dataset][method][zeta][edge_method] = G

    return graph_dict

In [ ]:
Graph_dict = generate_graph_dict_from_ttm_dict(TTM_dict, omega=0.1)

In [ ]:
with open("500N-KP-Crowd_graph_dict.pkl", "wb") as f:
    pickle.dump(Graph_dict, f)

In [ ]:
with open("500N-KP-Crowd_graph_dict.pkl", "rb") as f:
    graph_dict = pickle.load(f)

In [ ]:
def calculate_empirical_fc(G, runs=100, seed=2025):
    random.seed(seed)
    V = set(G.nodes())
    V_size = len(V)

    GC_nodes = set(max(nx.connected_components(G), key=len))
    GC_size = len(GC_nodes)
    P0 = GC_size / V_size

    fc_list = []
    all_traces= []

    for _ in range(runs):
        G_simulation = G.copy()
        node_list = list(G_simulation.nodes())
        random.shuffle(node_list)

        removed_nodes = set()
        trace = []

        fc_recorded = False

        for step, node in enumerate(node_list):
            G_simulation.remove_node(node)
            removed_nodes.add(node)

            remaining_GC_nodes = GC_nodes - removed_nodes

            if not remaining_GC_nodes:
                Pf = 0
            else:
                subgraph = G_simulation.subgraph(remaining_GC_nodes)
                cc_list = list(nx.connected_components(subgraph))
                if not cc_list:
                    Pf = 0
                else:
                    GC_now = max(cc_list, key=len)
                    Pf = len(GC_now) / V_size

                    largest_cc = max(nx.connected_components(G_simulation), key=len)
                    if len(largest_cc) > len(GC_now):
                        Pf = 0

            f = (step + 1) / V_size
            trace.append((f, Pf / P0))

            if not fc_recorded and (Pf / P0 == 0):
                fc_list.append(f)
                fc_recorded = True

        all_traces.append(trace)

    return np.mean(fc_list), all_traces

In [ ]:
def calculate_ratio(G):
    degrees = [d for _, d in G.degree()]
    k_avg = np.mean(degrees)
    k2_avg = np.mean(np.square(degrees))

    ratio = k2_avg / k_avg

    return ratio

In [ ]:
def calculate_ri(fc_empirical, fc_theoretical):
    if fc_empirical > fc_theoretical:
        return (fc_empirical - fc_theoretical) / (1 - fc_theoretical)
    else:
        return 0

In [ ]:
def evaluate_graph_dict(graph_dict, dataset_name, runs=100, seed=2025):
    records = []

    for method in graph_dict[dataset_name]:
        for zeta in graph_dict[dataset_name][method]:
            for edge_method, G in graph_dict[dataset_name][method][zeta].items():
                ratio = calculate_ratio(G)

                num_isolated_nodes = len(list(nx.isolates(G)))
                num_edges = G.number_of_edges()

                if ratio > 2:
                    fc_theoretical = 1 - 1 / (ratio - 1)
                    fc_empirical, all_traces = calculate_empirical_fc(G, runs=runs, seed=seed)
                    ri = calculate_ri(fc_empirical, fc_theoretical)
                else:
                    fc_theoretical = 0
                    fc_empirical = 0
                    ri = 0
                    all_traces = []

                records.append({
                    'Dataset': dataset_name,
                    'AKE Method': method,
                    'Zeta': zeta,
                    'Edge Measure': edge_method,
                    'Molloy-Reed Criterion': ratio,
                    'f_c (theoretical)': fc_theoretical,
                    'f_c (empirical)': fc_empirical,
                    'RI': ri,
                    'Isolated Nodes': num_isolated_nodes,
                    'Edge Count': num_edges,
                    'All Traces': all_traces
                })

    df = pd.DataFrame(records)
    return df

In [ ]:
results_df = evaluate_graph_dict(graph_dict, '500N-KP-Crowd', runs=100, seed=2025)

In [ ]:
results_df

In [ ]:
with open("500N-KP-Crowd_percolation_results.pkl", "wb") as f:
    pickle.dump(results_df, f)